# Pre-process and clean data

---
* Load the data sets (1 + 2)

---

In [ ]:
import pandas as pd

# Load Dataset 1: Only the 'question_text' column is relevant
dataset1 = pd.read_csv('dataset1.csv', delimiter=';', encoding='utf-8', usecols=['question_text'])

# Load Dataset 2: Only the 'question_title' and 'question_body' columns are relevant
dataset2 = pd.read_csv('dataset2.csv', delimiter=',', encoding='utf-8', usecols=['question_title', 'question_body'])

# Combine relevant columns from Dataset 2
dataset2['question_text'] = dataset2['question_title'].fillna('') + " " + dataset2['question_body'].fillna('')

# Drop the original columns after combining
dataset2 = dataset2[['question_text']]

# Progress Check-In
print("\n=== Initial Data Summary ===")
print(f"Dataset 1: {len(dataset1)} rows, columns: {dataset1.columns.tolist()}")
print(f"Dataset 2: {len(dataset2)} rows, columns: {dataset2.columns.tolist()}")

print("\n=== Dataset 1 Preview ===")
print(dataset1.head())

print("\n=== Dataset 2 Preview ===")
print(dataset2.head())



---
* Create progress check-in
* Create extraction of a sample set for review after each step

---

In [ ]:
# Progress check-in

def log_data_summary(data, step_name):
    print(f"\n=== Summary After Step: {step_name} ===")
    print(f"Number of rows: {len(data)}")
    print(f"Number of duplicate rows (based on 'question_text'): {data.duplicated(subset='question_text').sum()}")
    print(f"Number of empty rows in 'question_text': {data['question_text'].isnull().sum()}")
    print(f"Sample of 'question_text':\n{data['question_text'].head(5)}")


# Save a sample of 50 rows for review
def save_sample(data, step_name, sample_size=50):
    sample = data.sample(sample_size, random_state=42)
    sample.to_csv(f'sample_after_{step_name}.csv', index=False, encoding='utf-8')
    print(f"Sample saved for step: {step_name}")

---
* Concatenate data sets and handle duplicates

---

In [ ]:
# Combine datasets for de-duplication
combined_dataset = pd.concat([dataset1[['question_text']], dataset2[['question_text']]])

# Remove duplicates based on 'question_text'
combined_dataset.drop_duplicates(subset='question_text', inplace=True)

# Reset index after dropping duplicates
combined_dataset.reset_index(drop=True, inplace=True)

# Check-in
save_sample(combined_dataset, "deduplication")
log_data_summary(combined_dataset, "Deduplication - Combined Dataset")

---

* Add key to each question to ensure tracking and matching of questions with their context in later steps

---

In [ ]:
import uuid

# Add a unique key column using UUIDs
combined_dataset['question_id'] = [str(uuid.uuid4()) for _ in range(len(combined_dataset))]

# Save combinded dataset to a CSV file (to be able to compare with versions after cleaning steps)
combined_dataset.to_csv('combined_dataset.csv', index=False, encoding='utf-8')
print("Combined dataset saved to 'combined_dataset.csv'")

---

* Remove HTML tags

---

In [ ]:
import re

# Define function
def remove_html_tags(text):
    return re.sub(r'<[^>]*>', '', text)

# Apply to dataset
combined_dataset['question_text'] = combined_dataset['question_text'].apply(remove_html_tags)

# Check-in
save_sample(combined_dataset, "html_tag_removal")
log_data_summary(combined_dataset, "HTML Removal")

---

* Remove URLs

---

In [ ]:
# Define function
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text) # leave instances of "Gutefrage" in

# Apply to Dataset
combined_dataset['question_text'] = combined_dataset['question_text'].apply(remove_urls)

# Check-in
save_sample(combined_dataset, "url_removal")
log_data_summary(combined_dataset, "URL Removal")

---


* Normalise punctuation

---

In [ ]:
# Define function
def normalize_punctuation(text):
    # Replace underscores in numbers (e.g., 1_23 -> 1,23)
    #text = re.sub(r'(\d)_+(\d)', r'\1,\2', text)

    # Replace underscores in abbreviations (e.g., z_B -> z . B )
    text = re.sub(r'\b([A-Za-z])_+([A-Za-z])\b', r'\1 . \2', text)

    # Standardize quotation marks
    text = text.replace('“', '"').replace('”', '"').replace('„', '"').replace('‚', "'").replace('‘', "'").replace('´ ´', '"').replace('´', "'")

    return text


# Apply to Dataset
combined_dataset['question_text'] = combined_dataset['question_text'].apply(normalize_punctuation)

# Check-in
save_sample(combined_dataset, "punctuation_normalization")
log_data_summary(combined_dataset, "Punctuation Normalization")

---

* Remove emojis and unicode

---

In [ ]:
# Define function
def remove_non_printable_and_emojis(text):
    # Remove zero-width characters (e.g., <0x200b>)
    text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)

    # Remove emojis (cover a wide range of emoji Unicode blocks)
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002700-\U000027BF\U00002B50-\U00002B55]', '', text)

    # Remove Chinese characters (ranges: \u4E00-\u9FFF, \u3400-\u4DBF)
    text = re.sub(r'[\u4E00-\u9FFF\u3400-\u4DBF]', '', text)

    # Remove non-printable control characters
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

    # Retain Latin characters, punctuation, and symbols like +, -, (, ), [, ], etc.
    # Allow characters commonly used in text processing
    text = re.sub(r'[^\w\s.,$%&/#@°^:;€!?äöüßÄÖÜ+\-*/=<>()[\]{}]', '', text)

    return text

In [ ]:
# Apply to Dataset
combined_dataset['question_text'] = combined_dataset['question_text'].apply(remove_non_printable_and_emojis)

# Check-in
save_sample(combined_dataset, "emoji_unicode_removal")
log_data_summary(combined_dataset, "Emoji and Unicode Removal")

---

* Anonymise users

---

In [ ]:
# List of common German greetings and sign-offs (case insensitive)
sign_off_regex = r"(?i)((liebe(n|r)?|viele|beste(n)?|herzliche(n)?|mit freundliche(n|m)?) (grüße(n)?|gruß)|(?<!\w)lg(?!\w)|(?<!\w)mfg(?!\w)|(?<!\w)grüße(?!\w)|(?<!\w)gruß(?!\w))"

In [ ]:
# Define function
def anonymize_after_sign_off(text):
    # Search for the first occurrence of a sign-off pattern
    match = re.search(sign_off_regex, text)
    if match:
        # Preserve the part before the sign-off, append [ANONYMIZED] after the sign-off
        return text[:match.end()] + " [ANONYMIZED]"
    # Return the original text if no sign-off is found
    return text

In [ ]:
# Apply to Dataset
combined_dataset['question_text'] = combined_dataset['question_text'].apply(anonymize_after_sign_off)

# Check-in
save_sample(combined_dataset, "anonymized_sign_offs")
log_data_summary(combined_dataset, "Anonymization")

---

* Remove non-German material

---

In [ ]:
# Install library
!pip install langdetect

In [ ]:
from tqdm import tqdm
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure consistent results
DetectorFactory.seed = 42

# Define function
def detect_language_with_progress(text):
    try:
        # Detect the language of the text
        return detect(text)
    except LangDetectException:
        # Return 'unknown' if detection fails
        return 'unknown'

# Add tqdm progress bar for monitoring
tqdm.pandas(desc="Detecting Language")
combined_dataset['detected_language'] = combined_dataset['question_text'].progress_apply(detect_language_with_progress)

# Filter out rows that are not in German
german_dataset = combined_dataset[combined_dataset['detected_language'] == 'de']

# Save non-German rows to a file
non_german_dataset = combined_dataset[combined_dataset['detected_language'] != 'de']
non_german_dataset.to_csv('non_german_rows.csv', index=False, encoding='utf-8')
print(f"Non-German rows saved to 'non_german_rows.csv'")

# Log results
print(f"Total rows before filtering: {len(combined_dataset)}")
print(f"Rows retained (German): {len(german_dataset)}")
print(f"Rows removed (Non-German): {len(non_german_dataset)}")

# Drop temporary column after filtering
combined_dataset = german_dataset.drop(columns=['detected_language'])

# Check-in
save_sample(combined_dataset, "language_filtered")
log_data_summary(combined_dataset, "Language Filtering")

---

* Remove duplicates one more time
* Ensure the data set is ready for extraction and labelling

---

In [ ]:
# Remove duplicate rows
combined_dataset.drop_duplicates(subset='question_text', inplace=True)
combined_dataset.reset_index(drop=True, inplace=True)

save_sample(combined_dataset, "after_second_deduplication")
log_data_summary(combined_dataset, "Second Deduplication")

---
* Export cleaned data
* Save the cleaned data for further processing and labelling

---

In [ ]:
# Save the DataFrame
combined_dataset.to_csv('cleaned_dataset.csv', index=False, encoding='utf-8')